In [1]:
def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a + b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagonal_units = [[r + c for r, c in zip(rows, cols)], [r + c for r, c in zip(rows[::-1], cols)]]
unitlist = row_units + column_units + square_units + diagonal_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)



In [2]:
#-- utils

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return


# WARNING! We've modified this function to return '123456789' instead of '.' for boxes with no value.
# Look at the explanation above in the text.
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))



In [42]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    #--- create a list, doubles, of all boxes with double values
    doubles = [box for box in values if len(values[box]) == 2]
    print("doubles:", doubles)
    
    while len(doubles) > 1:

        #--- get the box from the list and see if we can find a twin
        box = doubles.pop()
        
        #--- find twin of box from the rest of the other doubles
        for other_double in doubles:
            #--- examine each unit for a twin, other_double, of box
            for unit in unitlist:
                #--- other_double is a twin if it is in the same unit and has the same value
                if box in unit and other_double in unit and values[box] == values[other_double]:
                    #--- other_double is twin of box!
                    #--- remove the values of the box from the other boxes in the same unit
                    for v in values[box]:
                        #--- the other boxes are from the same unit and have at least 3 values
                        other_boxes = [b for b in unit if b not in [box, other_double]] #--- and len(values[b]) > 2]
                        #--- remove the value, v from each of the otherboxes
                        for o in other_boxes:
                            values[o] = values[o].replace(v, '')

    return values                    


def naked_twins_try3(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    doubles = [box for box in values if len(values[box]) == 2]
    
    while len(doubles) > 1:
        print ("doubles", doubles)
        box = doubles.pop(0)
        for d in doubles:
            units_with_twin = [u for u in unitlist if box in u and d in u and values[box] == values[d]]
            if len(units_with_twin) == 0:
                continue
            print ("units with twin:", box, ":", units_with_twin)
            for u in units_with_twin:
                print("unit", u)
                other_boxes = [o for o in u if o not in [box, d] and len(values[o]) > 2]
                print("other_boxes", other_boxes)
                if len(other_boxes) == 0:
                    continue
                for v in values[box]:
                    for o in other_boxes:
                        values[o] = values[o].replace(v, '')

    return values                    

def naked_twins_old(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    #-- get all boxes with double values
    doubles = [box for box in values if len(values[box]) == 2]
    
    while len(doubles) > 1:
        
        #--- get first box with double value
        box = doubles.pop(0)

        #--- list all units where the box has a twin (boxes are in same unit and have same value)
        for d in doubles:
            units_with_twin = [u for u in unitlist if box in u and d in u and values[box] == values[d]]
            if len(units_with_twin) == 0:
                continue
            #--- units with twins of the box found!
            #--- for each unit, list the other boxes
            for u in units_with_twin:
                print("unit", u)
                other_boxes = [o for o in u if o not in [box, d] and len(values[o]) > 2]
                print("other_boxes", other_boxes)
                if len(other_boxes) == 0:
                    continue
                for v in values[box]:
                    for o in other_boxes:
                        values[o] = values[o].replace(v, '')

    return values                    



test_values = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}

test_values2 = {"G7": "1234568", "G6": "9", "G5": "35678", "G4": "23678", "G3":
"245678", "G2": "123568", "G1": "1234678", "G9": "12345678", "G8":
"1234567", "C9": "13456", "C8": "13456", "C3": "4678", "C2": "68",
"C1": "4678", "C7": "13456", "C6": "368", "C5": "2", "A4": "5", "A9":
"2346", "A8": "2346", "F1": "123689", "F2": "7", "F3": "25689", "F4":
"23468", "F5": "1345689", "F6": "23568", "F7": "1234568", "F8":
"1234569", "F9": "1234568", "B4": "46", "B5": "46", "B6": "1", "B7":
"7", "E9": "12345678", "B1": "5", "B2": "2", "B3": "3", "C4": "9",
"B8": "8", "B9": "9", "I9": "1235678", "I8": "123567", "I1": "123678",
"I3": "25678", "I2": "123568", "I5": "35678", "I4": "23678", "I7":
"9", "I6": "4", "A1": "2468", "A3": "1", "A2": "9", "A5": "3468",
"E8": "12345679", "A7": "2346", "A6": "7", "E5": "13456789", "E4":
"234678", "E7": "1234568", "E6": "23568", "E1": "123689", "E3":
"25689", "E2": "123568", "H8": "234567", "H9": "2345678", "H2":
"23568", "H3": "2456789", "H1": "2346789", "H6": "23568", "H7":
"234568", "H4": "1", "H5": "35678", "D8": "1235679", "D9": "1235678",
"D6": "23568", "D7": "123568", "D4": "23678", "D5": "1356789", "D2":
"4", "D3": "25689", "D1": "123689"}

test_values3= {"G7": "2345678", "G6": "1236789", "G5": "23456789", "G4": "345678",
"G3": "1234569", "G2": "12345678", "G1": "23456789", "G9": "24578",
"G8": "345678", "C9": "124578", "C8": "3456789", "C3": "1234569",
"C2": "1234568", "C1": "2345689", "C7": "2345678", "C6": "236789",
"C5": "23456789", "C4": "345678", "E5": "678", "E4": "2", "F1": "1",
"F2": "24", "F3": "24", "F4": "9", "F5": "37", "F6": "37", "F7": "58",
"F8": "58", "F9": "6", "B4": "345678", "B5": "23456789", "B6":
"236789", "B7": "2345678", "B1": "2345689", "B2": "1234568", "B3":
"1234569", "B8": "3456789", "B9": "124578", "I9": "9", "I8": "345678",
"I1": "2345678", "I3": "23456", "I2": "2345678", "I5": "2345678",
"I4": "345678", "I7": "1", "I6": "23678", "A1": "2345689", "A3": "7",
"A2": "234568", "E9": "3", "A4": "34568", "A7": "234568", "A6":
"23689", "A9": "2458", "A8": "345689", "E7": "9", "E6": "4", "E1":
"567", "E3": "56", "E2": "567", "E8": "1", "A5": "1", "H8": "345678",
"H9": "24578", "H2": "12345678", "H3": "1234569", "H1": "23456789",
"H6": "1236789", "H7": "2345678", "H4": "345678", "H5": "23456789",
"D8": "2", "D9": "47", "D6": "5", "D7": "47", "D4": "1", "D5": "36",
"D2": "9", "D3": "8", "D1": "36"}

display (test_values3)
print("-----")
display (naked_twins(test_values3))
    

 2345689   234568     7    |  34568      1      23689  |  234568   345689    2458  
 2345689  1234568  1234569 |  345678  23456789  236789 | 2345678  3456789   124578 
 2345689  1234568  1234569 |  345678  23456789  236789 | 2345678  3456789   124578 
---------------------------+---------------------------+---------------------------
    36       9        8    |    1        36       5    |    47       2        47   
   567      567       56   |    2       678       4    |    9        1        3    
    1        24       24   |    9        37       37   |    58       58       6    
---------------------------+---------------------------+---------------------------
 23456789 12345678 1234569 |  345678  23456789 1236789 | 2345678   345678   24578  
 23456789 12345678 1234569 |  345678  23456789 1236789 | 2345678   345678   24578  
 2345678  2345678   23456  |  345678  2345678   23678  |    1      345678     9    
-----
doubles: ['F2', 'F3', 'F5', 'F6', 'F7', 'F8', 'E3', 'D9', 'D7', 'D5', 

In [60]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    #-- get all boxes with double values
    doubles = [box for box in values if len(values[box]) == 2]
    
    while len(doubles) > 1:

        box = doubles.pop(0)
        
        #--- compare remaining boxes in doubles if it is a twin
        for d in doubles:

            #--- check if d is a row twin of box
            #--- note that box[0] specifies the row
            if (box[0] == d[0]) and (values[box] == values[d]):

                #--- found a row twin!
                #--- find other boxes in the row that have 3 or more numbers
                boxes_in_row = [box[0] + c for c in cols if len(values[box[0] + c]) > 2]
                
                #--- remove values v in other boxes on same row
                if len(boxes_in_row) > 0:
                    for v in values[box]:
                        #--- remove value v from each of the other boxes on the same row
                        for b in boxes_in_row:
                            values[b] = values[b].replace(v, '')
                    
            #--- check if d is a col twin of box
            #--- note that box[1] specifies the col
            elif (box[1] == d[1]) and (values[box] == values[d]):

                #--- found a col twin!
                #--- find other boxes in the col that have 3 or more numbers
                boxes_in_col = [r + box[1] for r in rows if len(values[r + box[1]]) > 2]
                
                #--- remove values v in other boxes on same row
                if len(boxes_in_col) > 0:
                    for v in values[box]:
                        #--- remove value v from each of the other boxes on the same row
                        for b in boxes_in_col:
                            values[b] = values[b].replace(v, '')

    return values        

test_values = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}


display (test_values)
print("")
print("----------------")
print("")

display (naked_twins(test_values))


  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  

----------------

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17 

In [47]:
assignments = []

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a + b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagonal_units = [[r + c for r, c in zip(rows, cols)], [r + c for r, c in zip(rows[::-1], cols)]]
unitlist = row_units + column_units + square_units + diagonal_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """

    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    
    #--- create a list, doubles, of all boxes with double values
    doubles = [box for box in values if len(values[box]) == 2]
    
    while len(doubles) > 1:

        #--- get the box from the list and see if we can find a twin
        box = doubles.pop()
        
        #--- find twin of box from the rest of the other doubles
        for other_double in doubles:
            #--- examine each unit for a twin, other_double, of box
            for unit in unitlist:
                #--- other_double is a twin if it is in the same unit and has the same value
                if box in unit and other_double in unit and values[box] == values[other_double]:
                    #--- other_double is twin of box!
                    #--- remove the values of the box from the other boxes in the same unit
                    for v in values[box]:
                        #--- the other boxes are from the same unit and have at least 3 values
                        other_boxes = [b for b in unit if b not in [box, other_double]] #--- and len(values[b]) > 2]
                        #--- remove the value, v from each of the otherboxes
                        for o in other_boxes:
                            assign_value(values, o, values[o].replace(v, ''))

    return values                        

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a + b for a in A for b in B]

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    labels = cross(rows, cols)
    
    return {k:v if not v == "." else '123456789' for (k, v) in zip(labels, grid)}


def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    #--- taken from the udacity quiz
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    labels = list(values.keys())
    
    for box in values:
        box_value = values[box]
        if len(box_value) == 1:
            #--- box is solved, find all peers and eliminate the box value from peers
            for peer_box in peers[box]:
                #--- remove box_value from peer by replacing with empty character
                peer_value = values[peer_box]
                peer_value = peer_value.replace(box_value, "")
                #--- updated the board with the updated value of the peer
                assign_value (values, peer_box, peer_value)

    return values

def only_choice(values):
    for unit in unitlist:
        for number in '123456789':
            #--- find which boxes in the unit list have the number
            boxes_having_number = [box for box in unit if number in values[box]]
            
            #--- if there is only one box with that number
            #--- then that is the only only choice for that box
            if len (boxes_having_number) == 1:
                #--- values[boxes_having_number[0]] = number
                assign_value (values, boxes_having_number[0], number)

    return values
    
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        
        #--- this was pretty much taken from the quiz, but we added the naked_twins strategy
        #--- use the naked_twins strategy
        values = naked_twins(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    if values is False:
        return False
        
    if all (len(values[box]) == 1 for box in boxes):
        #--- all values hav only 1 number solved
        return values
    
    # Choose one of the unfilled squares with the fewest possibilities
    lowest_len = 9
    best_box = ''
    for box in boxes:
        box_len = len(values[box])
        if box_len < lowest_len and box_len > 1:
            best_box = box
            l = box_len
            
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for number in values[best_box]:
        new_values = values.copy()
        assign_value (new_values, best_box, number)
        search_result = search(new_values)
        if search_result:
            return search_result
        #--- else try the next number from values[best_box]
        
    return
        
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    sudoku = grid_values(grid)
    
    display (sudoku)
    
    search_result = search(sudoku)

    if search_result:
        return search_result
    else:
        return False
    


In [48]:
diag_sudoku_grid = '9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'
display(solve(diag_sudoku_grid))



    9     123456789     1     |123456789 123456789 123456789 |123456789     8     123456789 
    8     123456789     5     |123456789     7     123456789 |123456789     4     123456789 
    2     123456789     4     |123456789 123456789 123456789 |123456789     6     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     7     |123456789 123456789 123456789 |123456789 123456789 123456789 
    5     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |    8         3     123456789 
------------------------------+------------------------------+------------------------------
    3     123456789 123456789 |    6     123456789 123456789 |123456789 123456789 123456789 
123456789     9     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678